<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Extracting-tweets-from-SQL-database" data-toc-modified-id="Extracting-tweets-from-SQL-database-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting tweets from SQL database</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Word2vec" data-toc-modified-id="Word2vec-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Word2vec</a></span></li></ul></li></ul></li><li><span><a href="#Topic-Modeling" data-toc-modified-id="Topic-Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Topic Modeling</a></span></li></ul></div>

# Libraries 

    The following Libraries were used in the development of this project:

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy
import nltk
import pyodbc
from nltk.tokenize import RegexpTokenizer, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
import json 

import warnings
warnings.filterwarnings("ignore")


# Data Preprocessing

The data preprocessing consist of different stages:

In [61]:
import nltk 


'''Natural Language Processing libraries'''
import nltk 
import gensim
import regex as re
import spacy
from spacy.lang.en import English
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
import re, string, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from gensim.models import FastText
from gensim.models import Word2Vec



class Document:
    """ Retrieve the narratives from the DataFrame and respectively
        store and pre-process it. 
        
        :param df: DataFrame including the reports and the predictor variable. 
        
        
        :ivar data: Stores the DataFrame.
        :ivar text: Stores the narratives as string.
        :ivar corpus: Stores the pre-processed text.
    """
    
    
    def __init__(self, df):
        self.data = df
        self.text = df["text"].astype(str)
        self.textPreProcessing()
        
        
    def remove_non_ascii(self, words):
        """Remove non-ASCII characters from list of tokenized words
        
        :param words:  List of words to be transformed when removing non_ascii characters.
        
        :return new_words: List of words after the transformation of removed non_ascii characters.
        
        """
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words


    def remove_punctuation(self, words):
        """Remove punctuation from list of tokenized words
        
        :param words:  List of words that will get remove their punctuations, if any. 
        
        :return new_words: List of transformed words.
        
        
        """
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words


    def stem_words(self, words):
        """Stem words in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective stems.
        
        
        """
        
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems
    
    def lemmatize_verbs(self, words):
        """Lemmatize verbs in list of tokenized words
        
        :param words:  List of words to be processed. 
        
        :return new_words: List of the received words respective lemmas.
        
        """
        
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas
    
    
    
    def remove_stopwords(self, words):
        """Remove common words that have no meaning or importance in the sentence.

        :param words:  List of words to be processed and get stop words removed.. 

        :return new_words: List of words with the stop words already removed."""
            
        
        stop_words = set(stopwords.words('spanish')) 
        print(stop_words)
        
        for word in stop_words:
            if word in words:
                words.remove(word)
                
        return words


    
    def normalize(self, words):
        words = self.remove_non_ascii(words)
        words = self.remove_stopwords(words)
        words = self.remove_punctuation(words)
        words = self.lemmatize_verbs(words)
        return words
    
    
    def textPreProcessing(self):
        """Pre-process the text, normalize and clean it.
        The function stores the cleaned text in the self.data
        attribute. """
      #  p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)


        clean_text = []

        for narrative in self.text:
          #  sentence = p.clean(narrative)
           # sentence = re.sub('RT @[\w_]+:', '', sentence)

            #sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
            sentence = word_tokenize(narrative)
            sentence = self.normalize(sentence)
                
                
            clean_text.append(sentence)
            
            
                    
        print(len(self.text), len(clean_text))
        self.data["clean_text"] = clean_text

In [59]:
def concatTweets(table):
    tweets = ''
    c = 0
    for row in df.tweets:
        if(c == 0):
            c+= 1
            tweets = pd.DataFrame(json.loads(row))
        else:
            tweets = pd.concat([tweets, pd.DataFrame(json.loads(row))], axis=0, ignore_index=True)
    
    
    return tweets 

# Extracting tweets from SQL database

In [26]:
server = 'sqldatamining.database.windows.net'
database = 'SNA'
username = 'UserAdmin'
password = 'Machomen123'   
driver= '{ODBC Driver 13 for SQL Server}'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER=sqldatamining.database.windows.net,1433', user='UserAdmin' , password='Machomen123', database='SNA')




In [27]:
cursor = cnxn.cursor()

In [28]:
rows = cursor.execute("SELECT * from dbo.Users1").fetchmany(10)

In [29]:
df = pd.DataFrame((tuple(t) for t in rows), columns=['id',
 'author',
 'created_at',
 'location',
 'description',
 'verified',
 'followers',
 'following',
 'favourites_count',
 'statuses_count',
 'lang',
 'tweets',
 'following_json',
 'followers_json']) 


In [30]:
df.head()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
0,2284857094,themoonisironic,2014-01-16 01:10:33,‘97 // they/them // ????????,"where there is hope, there are trials // art: ...",False,101,516,121166,36377,None,"[{""id"": 1356030533780824069, ""created_at"": ""20...","[{""id"": 1387079251476815875, ""author"": ""rikuno...","[{""id"": 1384297303129825284, ""author"": ""bts123..."
1,834324620,bbelita23,2012-09-19 23:21:48,"spooky town, PR",just put it out into the universe,False,670,443,33428,92917,None,"[{""id"": 1356026926134140928, ""created_at"": ""20...","[{""id"": 2513536418, ""author"": ""Kvn_Astacio"", ""...","[{""id"": 1289669319526420482, ""author"": ""keviin..."
2,175521394,Stormfocus18,2010-08-06 21:03:08,Puerto Rico_Island,"Loving #MewGulf and #KristSingto, amante de la...",False,199,187,6497,38945,None,[],"[{""id"": 1387677517822054400, ""author"": ""tinest...","[{""id"": 1387677517822054400, ""author"": ""tinest..."
3,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
4,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]


In [35]:
x = concatTweets(df)

x.tail()

,id,created_at,text,coordinates,geo,user,retweet_count,favorite_count
3886,1343697010101248001,2020-12-28T23:15:06,RT @luisfe23: Villano Antillano le doy menos d...,None,None,Angelicv_G,38,0
3887,1343696908632645633,2020-12-28T23:14:42,RT @marinaafd15: si tu hijx te dice que quiere...,None,None,Angelicv_G,21061,0
3888,1343694262869876742,2020-12-28T23:04:11,RT @DonVictorRamos: Villano Antillano es unx d...,None,None,Angelicv_G,22,0
3889,1343652117022650372,2020-12-28T20:16:43,RT @sinbuscarme: Artistas como @villanomalandr...,None,None,Angelicv_G,40,0
3890,1343652117022650372,2020-12-28T20:16:43,RT @sinbuscarme: Artistas como @villanomalandr...,None,None,Angelicv_G,40,0


In [43]:
dir(p)

['PyProcessorCapture',
 'Snippet',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'bcolors',
 'namedtuple',
 'process_file',
 'process_inline_snippet',
 'process_repeat_statements',
 'process_snippet',
 'process_source',
 're',
 'sys',
 'traceback']

In [54]:
import preprocessor as p
p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)
p.clean('Preprocessor í is #awesome 👍 https://github.com/s/preprocessor')

AttributeError: module 'preprocessor' has no attribute 'set_options'

In [53]:
!pip install tweet_preprocessor

In [56]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')


texto = pd.DataFrame(tuits, columns=["text"])

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\afabi/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [57]:
nltk.download('stopwords')
nltk.download('wordnet')
documento = Document(texto)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\afabi/nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


AttributeError: module 'preprocessor' has no attribute 'set_options'

In [502]:
documento.data.head()

,text,clean_text
0,Nada más cierto! 🤦🏻‍♀️🤦🏻‍♀️🤦🏻‍♀️,"[Nada, mas, cierto]"
1,Llamen a mi jefe diganle que la lluvia no me d...,"[Llamen, jefe, diganle, lluvia, deja, ir, trab..."
2,:,[]
3,": , , Pescao! Pescao!","[Pescao, Pescao]"
4,Así está bien o todavía no estas tan sorprendi...,"[Asi, bien, todavia, tan, sorprendido, El, gol..."


### Word2vec

In [356]:
model = Word2Vec(documento.data["clean_text"], min_count=0, workers=20, window=2,  alpha=0.02, hs=1)



In [503]:
my_dict = dict({})
for idx, key in enumerate(model.wv.key_to_index):
    my_dict[key] = model.wv[key]

In [504]:
vectores = []
for a in documento.data.clean_text:
    t  = []
    for word in a:
        try:
            t.append(my_dict[word])
        except:
            continue
    
    vectores.append(t)
        
documento.data["vectorized_text"] = vectores
documento.data.tail()

,text,clean_text,vectorized_text
113,": El ajusta la magnitud a , en el límite de pr...","[El, ajusta, magnitud, limite, producir, tsunami]","[[-0.017272485, 0.008652826, 0.007515971, 0.00..."
114,: “ Van a seguir ocurriendo eventos fuertes. H...,"[Van, seguir, ocurriendo, eventos, fuertes, Ha...","[[-0.0027897793, -0.0016228552, -0.007687815, ..."
115,: Casa colapsada en Yauco.,"[Casa, colapsada, Yauco]","[[0.008535036, 0.013252284, 0.00035941493, 0.0..."
116,: “Las réplicas podrían ser de magnitud mayor ...,"[Las, replicas, podrian, ser, magnitud, mayor,...","[[0.0029013832, -0.0027108788, -0.0015446608, ..."
117,": PRELIMINAR -01-06 :58:03 No hay Aviso, Adver...","[PRELIMINAR, 06, 5803, No, Aviso, Advertencia,...","[[-0.0127012255, 0.0040803654, 0.009027934, -0..."


In [505]:
df.tail()

,id,author,created_at,location,description,verified,followers,following,favourites_count,statuses_count,lang,tweets,following_json,followers_json
5,1685491040,CoraimaINegron,2013-08-20 10:48:59,Puerto Rico,"“El karma te lo devolverá todo, excepto a la m...",False,64,203,1336,1565,None,"[{""id"": 1283388114661244934, ""created_at"": ""20...","[{""id"": 299932350, ""author"": ""DMcIntyreWWE"", ""...","[{""id"": 713749428935462916, ""author"": ""DavidRo..."
6,1388202983964520449,Personn34091581,2021-04-30 18:46:05,Costa brava,,False,0,2,9,7,None,[],"[{""id"": 560803492, ""author"": ""UrbanLePharaon"",...",[]
7,908687401123553280,rsantanafonseca,2017-09-15 13:42:13,"COPU, UPRRP","19 (+3) | ?????? | Pop culture enthusiast, soc...",False,891,620,121676,124488,None,[],"[{""id"": 755882683, ""author"": ""_vidalysrms"", ""c...","[{""id"": 1344047685716815872, ""author"": ""LeQuee..."
8,1346688657365889024,SJCiudadCapital,2021-01-06 05:23:04,,Cuenta Oficial de la Ciudad Capital de Puerto ...,False,2915,183,291,1157,None,"[{""id"": 1355954464436473856, ""created_at"": ""20...","[{""id"": 1309256724830969857, ""author"": ""estefa...","[{""id"": 1271152649304518657, ""author"": ""JCruz_..."
9,521836439,Angelicv_G,2012-03-12 01:12:16,Puerto Rico,UT • yo no sé que poner aquí??,False,962,519,115665,130883,None,"[{""id"": 1356041673227313154, ""created_at"": ""20...","[{""id"": 1192074658994180096, ""author"": ""Nsnili...","[{""id"": 1383464766283280389, ""author"": ""geegee..."


In [62]:
c = 0
for x in df.tweets:
    tweets = pd.DataFrame(json.loads(x))
    c+=1 
    if(c==2):
        break
    

    
    
z = Document(tweets)


textos  = []
for a in z.data.clean_text.astype(str):
    for x in a:
        textos.append(a)
        


{'hay', 'algo', 'habríais', 'habido', 'vuestro', 'tuviésemos', 'teníais', 'estuvierais', 'para', 'todo', 'estarás', 'ellos', 'algunos', 'muchos', 'será', 'tu', 'suyas', 'había', 'seréis', 'o', 'sí', 'tened', 'sean', 'tendrán', 'estuvimos', 'fuiste', 'habrá', 'habidas', 'qué', 'tuyos', 'suyos', 'tuvieron', 'te', 'éramos', 'tuvo', 'tenida', 'teníamos', 'tú', 'fuese', 'uno', 'ni', 'hubieseis', 'tendríais', 'fuéramos', 'tenga', 'también', 'sentid', 'antes', 'seáis', 'poco', 'tiene', 'fuimos', 'tanto', 'en', 'hayan', 'eres', 'estada', 'esta', 'ti', 'nuestras', 'tienes', 'nosotros', 'tuvieseis', 'hube', 'hasta', 'serían', 'al', 'estamos', 'estoy', 'sus', 'estén', 'he', 'vosotras', 'seré', 'estuviese', 'me', 'que', 'mía', 'otro', 'has', 'tuyas', 'vuestra', 'estaremos', 'habrías', 'mucho', 'algunas', 'mías', 'hubo', 'quien', 'tuya', 'cual', 'sentidos', 'estuvo', 'habían', 'suyo', 'estará', 'tendrá', 'esto', 'les', 'tendréis', 'seas', 'mío', 'estuvieron', 'el', 'tenido', 'hubierais', 'estuviste

{'hay', 'algo', 'habríais', 'habido', 'vuestro', 'tuviésemos', 'teníais', 'estuvierais', 'para', 'todo', 'estarás', 'ellos', 'algunos', 'muchos', 'será', 'tu', 'suyas', 'había', 'seréis', 'o', 'sí', 'tened', 'sean', 'tendrán', 'estuvimos', 'fuiste', 'habrá', 'habidas', 'qué', 'tuyos', 'suyos', 'tuvieron', 'te', 'éramos', 'tuvo', 'tenida', 'teníamos', 'tú', 'fuese', 'uno', 'ni', 'hubieseis', 'tendríais', 'fuéramos', 'tenga', 'también', 'sentid', 'antes', 'seáis', 'poco', 'tiene', 'fuimos', 'tanto', 'en', 'hayan', 'eres', 'estada', 'esta', 'ti', 'nuestras', 'tienes', 'nosotros', 'tuvieseis', 'hube', 'hasta', 'serían', 'al', 'estamos', 'estoy', 'sus', 'estén', 'he', 'vosotras', 'seré', 'estuviese', 'me', 'que', 'mía', 'otro', 'has', 'tuyas', 'vuestra', 'estaremos', 'habrías', 'mucho', 'algunas', 'mías', 'hubo', 'quien', 'tuya', 'cual', 'sentidos', 'estuvo', 'habían', 'suyo', 'estará', 'tendrá', 'esto', 'les', 'tendréis', 'seas', 'mío', 'estuvieron', 'el', 'tenido', 'hubierais', 'estuviste

{'hay', 'algo', 'habríais', 'habido', 'vuestro', 'tuviésemos', 'teníais', 'estuvierais', 'para', 'todo', 'estarás', 'ellos', 'algunos', 'muchos', 'será', 'tu', 'suyas', 'había', 'seréis', 'o', 'sí', 'tened', 'sean', 'tendrán', 'estuvimos', 'fuiste', 'habrá', 'habidas', 'qué', 'tuyos', 'suyos', 'tuvieron', 'te', 'éramos', 'tuvo', 'tenida', 'teníamos', 'tú', 'fuese', 'uno', 'ni', 'hubieseis', 'tendríais', 'fuéramos', 'tenga', 'también', 'sentid', 'antes', 'seáis', 'poco', 'tiene', 'fuimos', 'tanto', 'en', 'hayan', 'eres', 'estada', 'esta', 'ti', 'nuestras', 'tienes', 'nosotros', 'tuvieseis', 'hube', 'hasta', 'serían', 'al', 'estamos', 'estoy', 'sus', 'estén', 'he', 'vosotras', 'seré', 'estuviese', 'me', 'que', 'mía', 'otro', 'has', 'tuyas', 'vuestra', 'estaremos', 'habrías', 'mucho', 'algunas', 'mías', 'hubo', 'quien', 'tuya', 'cual', 'sentidos', 'estuvo', 'habían', 'suyo', 'estará', 'tendrá', 'esto', 'les', 'tendréis', 'seas', 'mío', 'estuvieron', 'el', 'tenido', 'hubierais', 'estuviste

{'hay', 'algo', 'habríais', 'habido', 'vuestro', 'tuviésemos', 'teníais', 'estuvierais', 'para', 'todo', 'estarás', 'ellos', 'algunos', 'muchos', 'será', 'tu', 'suyas', 'había', 'seréis', 'o', 'sí', 'tened', 'sean', 'tendrán', 'estuvimos', 'fuiste', 'habrá', 'habidas', 'qué', 'tuyos', 'suyos', 'tuvieron', 'te', 'éramos', 'tuvo', 'tenida', 'teníamos', 'tú', 'fuese', 'uno', 'ni', 'hubieseis', 'tendríais', 'fuéramos', 'tenga', 'también', 'sentid', 'antes', 'seáis', 'poco', 'tiene', 'fuimos', 'tanto', 'en', 'hayan', 'eres', 'estada', 'esta', 'ti', 'nuestras', 'tienes', 'nosotros', 'tuvieseis', 'hube', 'hasta', 'serían', 'al', 'estamos', 'estoy', 'sus', 'estén', 'he', 'vosotras', 'seré', 'estuviese', 'me', 'que', 'mía', 'otro', 'has', 'tuyas', 'vuestra', 'estaremos', 'habrías', 'mucho', 'algunas', 'mías', 'hubo', 'quien', 'tuya', 'cual', 'sentidos', 'estuvo', 'habían', 'suyo', 'estará', 'tendrá', 'esto', 'les', 'tendréis', 'seas', 'mío', 'estuvieron', 'el', 'tenido', 'hubierais', 'estuviste

In [63]:
z.data.head()

,id,created_at,text,coordinates,geo,user,retweet_count,favorite_count,clean_text
0,1356026926134140928,2021-01-31T23:49:47,yikes 😬 https://t.co/pp9mcK3zhL,None,None,bbelita23,0,0,"[yikes, https, tcopp9mcK3zhL]"
1,1355947293153894400,2021-01-31T18:33:21,"RT @CcNeiko: I need Giveon , Frank Ocean &amp;...",None,None,bbelita23,11227,0,"[RT, CcNeiko, I, need, Giveon, Frank, Ocean, a..."
2,1355947277949530116,2021-01-31T18:33:18,RT @TheMonai: Your secret is always safe with ...,None,None,bbelita23,5591,0,"[RT, TheMonai, Your, secret, be, always, safe,..."
3,1355912548630081542,2021-01-31T16:15:17,no pido mucho solo un apartamento en NY 😔,None,None,bbelita23,0,2,"[pido, solo, apartamento, NY]"
4,1355912278726631430,2021-01-31T16:14:13,RT @shaistadeen: outta sight but on your mind....,None,None,bbelita23,13997,0,"[RT, shaistadeen, outta, sight, but, on, your,..."


# Topic Modeling 

In [508]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt

In [540]:
id2word = corpora.Dictionary(z.data.clean_text)

# Create Corpus
texts = z.data.clean_text

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [541]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [520]:
print(lda_model.print_topics())

[(0, '0.035*"tsunami" + 0.027*"Rico" + 0.027*"Puerto" + 0.027*"PRELIMINAR" + 0.025*"No" + 0.023*"Aviso" + 0.023*"Advertencia" + 0.023*"Islas" + 0.023*"Vigilancia" + 0.022*"M"'), (1, '0.013*"No" + 0.012*"PR" + 0.012*"Prof" + 0.012*"Ml" + 0.012*"Km" + 0.012*"Sentido" + 0.012*"Int" + 0.012*"Mag" + 0.012*"Max" + 0.011*"Aviso"'), (2, '0.025*"mayor" + 0.025*"magnitud" + 0.022*"Molinelli" + 0.013*"seguir" + 0.013*"ser" + 0.013*"seria" + 0.013*"un" + 0.013*"NO" + 0.011*"Jose" + 0.011*"eventos"'), (3, '0.024*"base" + 0.024*"Mas" + 0.024*"estadounidenses" + 0.017*"a" + 0.015*"La" + 0.014*"isla" + 0.012*"Reportan" + 0.012*"aereo" + 0.012*"hora" + 0.012*"soldados"'), (4, '0.026*"temblor" + 0.019*"Y" + 0.019*"Temblor" + 0.018*"Yauco" + 0.017*"colapsada" + 0.017*"Casa" + 0.017*"Caguas" + 0.017*"Cabo" + 0.017*"Toa" + 0.017*"Baja"')]


In [2]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

NameError: name 'pyLDAvis' is not defined

In [359]:
word2vec_model.wv.key_to_index

{'de': 0,
 'No': 1,
 'a': 2,
 'Aviso': 3,
 'Bayamon': 4,
 'Nuevo': 5,
 'Barrio': 6,
 'El': 7,
 'la': 8,
 'magnitud': 9,
 'Ml': 10,
 'Sentido': 11,
 'Km': 12,
 'Prof': 13,
 'Mag': 14,
 'PRELIMINAR': 15,
 'Max': 16,
 'Puerto': 17,
 'Rico': 18,
 'Int': 19,
 'temblor': 20,
 'PR': 21,
 'enero2020': 22,
 'PM': 23,
 'que': 24,
 'mas': 25,
 'tsunami': 26,
 'be': 27,
 'sabado': 28,
 'luz': 29,
 'servicio': 30,
 'hoy': 31,
 'La': 32,
 'Gracias': 33,
 'los': 34,
 'Wanda': 35,
 'Hoy': 36,
 'mayor': 37,
 'mejor': 38,
 'ser': 39,
 'el': 40,
 'vez': 41,
 'Que': 42,
 'Mayaguez': 43,
 'gente': 44,
 'fuerte': 45,
 'ver': 46,
 'si': 47,
 'l': 48,
 'Mas': 49,
 'cada': 50,
 'ir': 51,
 'chicharra': 52,
 'con': 53,
 'preliminar': 54,
 'Islas': 55,
 'Advertencia': 56,
 'Vigilancia': 57,
 'no': 58,
 'Los': 59,
 'ACTUALIZADA': 60,
 'temblores': 61,
 'pandemia': 62,
 'Lo': 63,
 'Enero': 64,
 'tan': 65,
 'espero': 66,
 'isla': 67,
 'Hola': 68,
 'cierto': 69,
 'Pescao': 70,
 'bien': 71,
 'vi': 72,
 'NO': 73,
 'ent